In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('covid19-global-forecasting-week-3/train.csv')
df_test = pd.read_csv('covid19-global-forecasting-week-3/test.csv')
df_train.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [3]:
df_test.head()

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-03-26
1,2,NaN,Afghanistan,2020-03-27
2,3,NaN,Afghanistan,2020-03-28
3,4,NaN,Afghanistan,2020-03-29
4,5,NaN,Afghanistan,2020-03-30


In [4]:
train = df_train
test = df_test

In [5]:
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)
train['Date'] = train['Date'].dt.strftime("%d%m").astype(int)
test['Date'] = test['Date'].dt.strftime("%d%m").astype(int)

In [6]:
train.Province_State.fillna('Data_NA',inplace=True)
test.Province_State.fillna('Data_NA',inplace=True)

In [7]:
Countries = train.Country_Region.unique()

In [8]:
Provinces = train.groupby('Country_Region')['Province_State'].unique()

In [17]:
len(Countries)*len(Provinces)

32400

In [44]:
from xgboost import XGBRegressor

class COVID_19:
    def __init__(self):
        self.count = 0
        self.Predictions = {'ForecastId':[],'ConfirmedCases':[],'Fatalities':[]}
    
    def Forecast_COVID_19(self,N=324):
        for country in Countries:
            for province in Provinces[country]:
                country_df = train[(train.Country_Region==country)]
                province_df = country_df[(country_df.Province_State==province)]

                X_train = province_df.Date.values.reshape(-1,1)
                y_trainC = province_df.ConfirmedCases.values
                y_trainF = province_df.Fatalities.values

                test_country_df = test[(test.Country_Region==country)]
                test_province_df = test_country_df[(test_country_df.Province_State==province)]
                X_test = test_province_df.Date.values.reshape(-1,1)

                y_predC = XGBRegressor(n_estimators=N).fit(X_train, y_trainC).predict(X_test)
                y_predF = XGBRegressor(n_estimators=N).fit(X_train, y_trainF).predict(X_test)

                FC_ID = np.array(test_province_df.ForecastId)

                for i,(j,k) in enumerate(zip(y_predC,y_predF)):
                    self.Predictions['ForecastId'].append(FC_ID[i])
                    self.Predictions['ConfirmedCases'].append(np.round(j))
                    self.Predictions['Fatalities'].append(np.round(k))

        df_pred = pd.DataFrame(self.Predictions)

        if self.count==0:
            df_pred.to_csv('submission.csv',index=False)
        if self.count>=1:
            df_pred.to_csv('submission_'+str(self.count)+'.csv',index=False)

        self.count+=1

In [45]:
COVID_Forecaster = COVID_19()
COVID_Forecaster.Forecast_COVID_19()
COVID_Forecaster.Forecast_COVID_19(1000)